In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [5]:
df.shape

(891, 12)

In [6]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df = df.loc[df.Embarked.notna(),['Survived', 'Pclass', 'Sex', 'Embarked']]

In [10]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [11]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  889 non-null    int64 
 1   Pclass    889 non-null    int64 
 2   Sex       889 non-null    object
 3   Embarked  889 non-null    object
dtypes: int64(2), object(2)
memory usage: 34.7+ KB


### since pclass should be the categorical column but we will remain itas numerical for now , we start by cross validating the model that predicts only using pclass, then will see ohe, pipeline and column transformer

In [16]:
X = df.loc[:, ['Pclass']]
y = df.Survived

In [17]:
X.shape

(889, 1)

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
logreg = LogisticRegression()
#different solvers

In [21]:
from sklearn.model_selection import cross_val_score

In [22]:
cross_val_score(logreg, X, y, cv = 5, scoring = 'accuracy').mean()

0.6783406335301212

In [24]:
#null accuracy - predicting the accuracy of the the most frequent class which is 0.61
y.value_counts(normalize = True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [25]:
#i want to add more features to my model and croos validate it by pipeline and encoding categories
from sklearn.preprocessing import OneHotEncoder

In [26]:
ohe = OneHotEncoder(sparse = False)

In [27]:
#ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [28]:
#ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [29]:
X = df.drop(['Survived'], axis = 1)

In [30]:
X.shape

(889, 3)

In [31]:
#column transformer
from sklearn.compose import make_column_transformer

In [32]:
#we use column transformer in our dataframe any time when we have features in our df that need different preprocessing
column_trans = make_column_transformer((OneHotEncoder(), ['Sex', 'Embarked']), remainder = 'passthrough')
#since we had applied ohe on sex and embarked but pclass was numerical so we it to be as it is hence we have to passthrough the remainder column
#we can add further thing into it as well may be wer want to do simple imputation on some column and some want to leave as it

In [33]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

# now making the pipeline

In [34]:
from sklearn.pipeline import make_pipeline
#pipelining is for chaining steps togethor

In [35]:
pipe = make_pipeline(column_trans, logreg)

In [36]:
cross_val_score(pipe, X, y, cv = 5, scoring = 'accuracy').mean()

0.7727924839713071

In [39]:
X_new = X.sample(8, random_state = 122)
X_new

,Pclass,Sex,Embarked
494,3,male,S
802,1,male,S
438,1,male,S
622,3,male,C
573,3,female,Q
315,3,female,S
806,1,male,S
476,2,male,S


In [40]:
pipe.fit(X,y)
#similar to model.fit except it does model fit as well as preprocessing

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [42]:
pipe.predict(X_new)
#just like model.fit except it runs the preprocessing steps on my new model

array([0, 0, 0, 0, 1, 1, 0, 0], dtype=int64)